In [1]:
#import torch
import pandas as pd
import numpy as np
import networkx as nx
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
import os
os.chdir('/home/lyz/PhaSepGAT/PSGAT/')
DATA_ROOT = './data'

In [5]:
import os
os.chdir('D:/lyz所有代码/PhaSepGAT/')
DATA_ROOT = './data'

In [6]:
SAVE_ROOT = './saves/INTEGRATE_ProSE80d_pos1neg1/'
preds = pd.read_pickle(f'{SAVE_ROOT}preds.pkl')
genes = pd.read_pickle(f'{SAVE_ROOT}genes.pkl')

N = 10

In [130]:
genename_to_uniprot = pd.read_pickle("./OtherVisualization/data/HUMAN_9606_idmapping_genename.pkl")
genename_to_uniprot_integrate = genename_to_uniprot.set_index(['Uniprot']).loc[list(set(genes)&set(genename_to_uniprot['Uniprot'].tolist())),:].reset_index()

genename_to_uniprot_dict = genename_to_uniprot_integrate.set_index(['ID']).to_dict()['Uniprot']
uniprot_to_genename_dict = genename_to_uniprot_integrate.set_index(['Uniprot']).to_dict()['ID']

In [7]:
phasepred_df = pd.read_pickle('./data/HUMAN_9606_PhaSePred_SaPS_PdPS.pkl')
entry2pdps_dict = phasepred_df.set_index('entry').to_dict()['PdPS-10fea']
entry2saps_dict = phasepred_df.set_index('entry').to_dict()['SaPS-10fea']

phasep_df = pd.read_pickle('./data/Label/human.condensate-formation.proteins-v1.pkl')
lt = set(phasep_df[phasep_df['source']=='literature']['uniprot_entry'])
pr = set(phasep_df[phasep_df['source']=='phasepred']['uniprot_entry'])
pr = pr - lt
dp = set(phasep_df[phasep_df['source']=='deepphase']['uniprot_entry'])
dp = dp - pr - lt

In [8]:
ppi = 'integrate'

ppi_path = os.path.join(
    DATA_ROOT,           
    f'PPIN/{ppi.upper()}.csv'
)
edges = pd.read_csv(ppi_path)
edges['C'] = np.sort(edges[['A', 'B']].values).sum(axis=1)

for i in range(N): 
    tmp = pd.read_pickle(f'{SAVE_ROOT}edge_cosine/model_{i}.pkl')
    edges[i] = tmp['cosim']
edges['pred'] = edges.iloc[:, 3:].mean(axis=1)

edges.loc[(edges['A'].isin(lt))&(edges['B'].isin(lt)), 'type'] = 'ps.ppi'
edges['type'] = edges['type'].fillna('nps.ppi')

In [9]:
for n in range(N):
    edges[['A', 'B', n]].to_csv(f'{SAVE_ROOT}edge_lists/weighted_{n}.txt', sep=' ', index=None, header=None)

edges[['A', 'B']].to_csv(f'{SAVE_ROOT}edge_lists/unweighted.txt', sep=' ', index=None, header=None)

edges[['A', 'B', 'pred']].to_csv(f'{SAVE_ROOT}edge_lists/weighted_average.txt', sep=' ', index=None, header=None)

# Ensemble clustering preprocess

In [10]:
from itertools import chain

def chainer(s, sep):
    return list(chain.from_iterable(s.str.split(sep)))

def split_items(dataframe, col_name, sep):
    lens = dataframe[col_name].str.split(sep).map(len)
    cols = dataframe.columns
    dict1 = {c: np.repeat(dataframe[c], lens) for c in cols}
    dict1[col_name] = chainer(dataframe[col_name], sep)
    return pd.DataFrame(dict1)

In [192]:
cl = pd.read_csv(f'./saves/INTEGRATE_ProSE80d_pos1neg1/clusterone_outputs/weighted_average.txt')

In [ ]:
#cl = pd.read_csv(f'./saves/INTEGRATE_ProSE80d_pos1neg1/clusterone_outputs/unweighted_d03.txt')

In [40]:
cl

,Cluster,Size,Density,Internal weight,External weight,Quality,P-value,Members
0,1,31,0.3021,140.500,758.1,0.14630,0.999874,Q9Y5B9 Q08945 Q96P70 Q9BTT0 P62805 Q01831 P499...
1,2,18,0.3814,58.360,142.8,0.24600,0.556608,P49848 P21675 Q9HBM6 Q15545 Q15542 Q9BR10 Q155...
2,3,24,0.2211,61.030,257.6,0.16650,0.991380,Q9HAH7 Q8IY57 Q86SE9 P61962 Q9BYE7 Q8N488 P783...
3,4,9,0.3027,10.900,276.5,0.03569,0.999713,Q9UHE5 O95214 P54849 O15529 O15552 Q9HDC5 E9PQ...
4,5,82,0.3011,999.900,5236.0,0.15620,0.958322,Q13586 P50402 P78310 Q8N144 Q96RD7 P51151 Q8WX...
...,...,...,...,...,...,...,...,...
2088,2089,6,0.3092,4.638,250.7,0.01735,0.977336,P02792 A2A3N6 Q9UQ07 Q0VDD8 Q9UJK0 Q6ZT52
2089,2090,6,0.3100,4.651,250.7,0.01740,0.977336,P02792 A2A3N6 Q9UQ07 Q0VDD8 Q9UJK0 Q9Y663
2090,2091,5,0.3345,3.345,130.3,0.02329,0.952654,Q9Y5Y6 Q9UBS3 O95672 P34096 A6NK58
2091,2092,5,0.3207,3.207,130.4,0.02233,0.952654,Q9Y5Y6 Q9UBS3 O95672 P34096 Q9NZV1


## Identifying PS clusters

In [191]:
phasepred_df = phasepred_df.set_index(['entry']).loc[genes,:]

KeyError: "None of ['entry'] are in the columns"

In [107]:
known_ps = pd.read_pickle("./data/Label/human.condensate-formation.proteins-v1.pkl")
known_ps =  known_ps[known_ps['source']=='literature']['uniprot_entry'].tolist()
known_ps

['Q08752',
 'Q96NY9',
 'Q13247',
 'Q8IZH2',
 'P63279',
 'Q9Y2W1',
 'Q495M9',
 'P51608',
 'O94818',
 'Q16637',
 'Q9Y580',
 'Q6ZN04',
 'Q9Y4E5',
 'O75554',
 'Q14011',
 'Q13283',
 'P06748',
 'Q96FC9',
 'Q14739',
 'Q9UKD2',
 'P11142',
 'Q9NPH5',
 'P43243',
 'Q8WV28',
 'Q15643',
 'P36873',
 'Q13435',
 'Q9HBD1',
 'Q9BPY3',
 'Q32P51',
 'P20226',
 'Q8WWY3',
 'Q9GZQ8',
 'Q5H9S7',
 'Q8NEJ9',
 'Q8TCF1',
 'Q99607',
 'Q9NR00',
 'Q9NRN9',
 'Q96FQ6',
 'O00571',
 'Q99459',
 'Q13823',
 'Q9NR21',
 'A4D1E9',
 'O60318',
 'P17480',
 'Q9H633',
 'P16401',
 'O75533',
 'Q8NI36',
 'O60885',
 'P27694',
 'Q8ND90',
 'Q9BSJ6',
 'Q14191',
 'Q6PJT7',
 'Q9NX58',
 'Q504Q3',
 'Q9Y3B2',
 'Q96RS0',
 'Q8NB46',
 'Q9HCE1',
 'Q9Y3Y2',
 'Q75N03',
 'Q9NRR4',
 'Q96AY2',
 'Q9Y4C8',
 'Q9BXS6',
 'Q15811',
 'O15381',
 'O75177',
 'A7MCY6',
 'Q9NPJ4',
 'Q16082',
 'P17066',
 'P55884',
 'Q96QA6',
 'Q8IXH7',
 'P00441',
 'Q9Y2V2',
 'Q9NSU2',
 'Q8TC56',
 'Q9NRA8',
 'Q9UHD2',
 'Q9BUT9',
 'O60828',
 'Q86WT6',
 'Q3ZCQ8',
 'Q00839',
 'O00422',

In [74]:
ps_rank_self = phasepred_df.set_index(['entry']).loc[genes,:].sort_values(by=['SaPS-10fea'],ascending=False).loc[:,['SaPS-10fea']]
ps_rank_other = phasepred_df.set_index(['entry']).loc[genes,:].sort_values(by=['PdPS-10fea'],ascending=False).loc[:,['PdPS-10fea']]

In [206]:
ps_protein_list = list(phasepred_df[(phasepred_df['SaPS-10fea']>0.5) | (phasepred_df['PdPS-10fea']>0.5)].index)
len(ps_protein_list)

4625

In [207]:
ps_protein_list=np.unique(ps_protein_list+known_ps).tolist()
len(ps_protein_list)

4845

In [113]:
def ps_enrich(Member, Entry):
        members = Member.split(' ')
        entries = Entry
        a = set(members) & set(entries)
        b = set(members) - set(entries)
        c = set(entries) - set(members)
        d = set(genes) - set(members) | set(entries)
        frac = len(a) / len(members)
        from scipy.stats import fisher_exact
        _, pval = fisher_exact([[len(a), len(b)], [len(c), len(d)]])
        return frac, pval

In [208]:
pvals, fracs = dict(), dict()
entrys = ps_protein_list
temp_ls1, temp_ls2 = list(), list()
for i in range(len(cl)):
    members = cl.loc[i,'Members']
    fra, p = ps_enrich(members, entrys)
    temp_ls1.append(p)
    temp_ls2.append(fra)
pvals['PS proteins'] = temp_ls1
fracs['PS proteins'] = temp_ls2

In [209]:
fracs_df = pd.DataFrame(fracs)
cols = ['Frac ' + c for c in fracs_df.columns]
fracs_df.columns = cols

pvals_df = pd.DataFrame(pvals)
cols = ['FDR ' + c for c in pvals_df.columns]
pvals_df.columns = cols
# FDR correction
from statsmodels.stats.multitest import fdrcorrection
for c in pvals_df.columns:
    _, fdrs = fdrcorrection(pvals_df[c])
    pvals_df[c] = fdrs

enrich_df = pd.concat([cl, pvals_df, fracs_df], axis=1)

In [219]:
ps_clusters = enrich_df[(enrich_df['FDR PS proteins']<0.05)|(enrich_df['Frac PS proteins']>0.5)]

In [220]:
len(ps_clusters)

493

In [227]:
ps_clusters

,Cluster,Size,Density,Internal weight,External weight,Quality,P-value,Members,FDR PS proteins,Frac PS proteins,Entry,Gene Name,Label
0,1,31,0.3021,140.500,758.10,0.14630,0.999874,Q9Y5B9 Q08945 Q96P70 Q9BTT0 P62805 Q01831 P499...,1.036222e-03,0.580645,Q08945,SSRP1,SSRP1(31)
1,3,24,0.2211,61.030,257.60,0.16650,0.991380,Q9HAH7 Q8IY57 Q86SE9 P61962 Q9BYE7 Q8N488 P783...,1.380849e-03,0.625000,Q8IY57,YAF2,YAF2(24)
2,5,82,0.3011,999.900,5236.00,0.15620,0.958322,Q13586 P50402 P78310 Q8N144 Q96RD7 P51151 Q8WX...,7.453350e-05,0.475610,Q9H3P7,ACBD3,ACBD3(82)
3,8,4,0.3167,1.900,22.65,0.05839,0.903035,Q9ULL5 Q8IWW6 Q9UPX8 Q9BYB0,3.507421e-02,1.000000,Q8IWW6,ARHGAP12,ARHGAP12(4)
4,9,4,0.3703,2.222,30.80,0.05416,0.992931,P45973 P23497 Q6N043 Q9H5I1,2.109341e-01,0.750000,P23497,SP100,SP100(4)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
488,2069,4,0.3148,1.889,55.97,0.02868,0.903035,Q9BW19 Q86XP3 Q9NPA8 P28324,2.109341e-01,0.750000,Q86XP3,DDX42,DDX42(4)
489,2078,6,0.3188,4.783,86.66,0.04624,0.967224,P35611 Q8N6D5 Q9UEY8 P35612 P32456 Q5JUQ0,1.952822e-01,0.666667,P35611,ADD1,ADD1_1(6)
490,2079,12,0.3006,19.840,274.80,0.06225,0.999761,Q86V42 Q9HAP6 Q8N3R9 Q9Y2J4 P35580 Q8IY63 P469...,8.810641e-02,0.583333,Q8IY63,AMOTL1,AMOTL1(12)
491,2082,32,0.3011,149.300,2772.00,0.05002,1.000000,Q96A22 Q03135 P78310 O75955 P06239 P07948 P299...,1.233763e-09,0.812500,Q8TEW0,PARD3,PARD3_4(32)


In [228]:
def cl_entry(x, feat='PdPS-10fea'):
    ls = x.split(' ')
    df = pd.DataFrame(ls)
    df['PdPS-10fea'] = df[0].map(entry2pdps_dict)
    df['SaPS-10fea'] = df[0].map(entry2saps_dict)
    e = df[df[feat]==df[feat].max()][0].item()
    return e

ps_clusters['Entry'] = ps_clusters['Members'].apply(lambda x: cl_entry(x))

C:\Users\wqswx\AppData\Local\Temp\ipykernel_17092\4044319039.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ps_clusters['Entry'] = ps_clusters['Members'].apply(lambda x: cl_entry(x))


In [229]:
ps_clusters['Gene Name'] = ps_clusters['Entry'].map(uniprot_to_genename_dict)

C:\Users\wqswx\AppData\Local\Temp\ipykernel_17092\3999640075.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ps_clusters['Gene Name'] = ps_clusters['Entry'].map(uniprot_to_genename_dict)


In [230]:
ps_clusters.index=range(len(ps_clusters))
already = []
labels = []
for i in tqdm(range(len(ps_clusters))):
    entry = ps_clusters.loc[i,'Gene Name']
    count = ps_clusters.loc[i,'Size']
    if entry not in already:
        label = f"{entry}({count})"
        already.append(entry)
        labels.append(label)
    else:
        label = f"{entry}_{already.count(entry)}({count})"
        already.append(entry)
        labels.append(label)

100%|██████████| 493/493 [00:00<00:00, 61349.71it/s]


In [231]:
ps_clusters['Label'] = labels

C:\Users\wqswx\AppData\Local\Temp\ipykernel_17092\1254480317.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ps_clusters['Label'] = labels


In [233]:
ps_clusters['mean_SaPS'] = [np.mean([entry2saps_dict[k] for k in i.split(' ')]) for i in ps_clusters['Members'].tolist()]
ps_clusters['mean_PdPS'] = [np.mean([entry2pdps_dict[k] for k in i.split(' ')]) for i in ps_clusters['Members'].tolist()]

C:\Users\wqswx\AppData\Local\Temp\ipykernel_17092\4045246981.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ps_clusters['mean_SaPS'] = [np.mean([entry2saps_dict[k] for k in i.split(' ')]) for i in ps_clusters['Members'].tolist()]
C:\Users\wqswx\AppData\Local\Temp\ipykernel_17092\4045246981.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ps_clusters['mean_PdPS'] = [np.mean([entry2pdps_dict[k] for k in i.split(' ')]) for i in ps_clusters['Members'].tolist()]


In [234]:
ps_clusters.head(1)

,Cluster,Size,Density,Internal weight,External weight,Quality,P-value,Members,FDR PS proteins,Frac PS proteins,Entry,Gene Name,Label,mean_SaPS,mean_PdPS
0,1,31,0.3021,140.5,758.1,0.1463,0.999874,Q9Y5B9 Q08945 Q96P70 Q9BTT0 P62805 Q01831 P499...,0.001036,0.580645,Q08945,SSRP1,SSRP1(31),0.378145,0.543313


In [235]:
ps_clusters.to_pickle("./saves/INTEGRATE_ProSE80d_pos1neg1/PS_Clusters_basedata(493).pkl")

In [187]:
ps_clusters.to_csv("E:/lyz_PhaGAT/代码整理/Fig5/PS_Clusters_basedata.csv")

## MLOs enrichment in Pred Clusters

In [140]:
mlo_raw = pd.read_excel('./data/phasepdbv2_1_mlolt_mloht.xlsx')
mlo = mlo_raw[(mlo_raw['organism']=='Homo sapiens')&(mlo_raw['entry']!='_')].reset_index(drop=True)
mlo['entry'] = mlo['entry'].apply(lambda x: x.split('-')[0])
mloset = mlo.groupby('MLO')['entry'].apply(lambda x: ' '.join(set(x))).to_frame().reset_index()

In [141]:
mloset.head()

,MLO,entry
0,Cajal body,P61956 P07332 P29376 P08922 P19338 Q6PHR2 Q152...
1,"Cajal body,Nucleolus",O95453 O60832 P22087
2,Centrosome,Q99996
3,DNA damage foci,Q5UIP0
4,Histone locus body,P49711 Q14207 Q9UKL3


In [142]:
mloset['size'] = mloset['entry'].apply(lambda x: len(set(x.split(' '))))
mloset.sort_values(by='size', ascending=False).reset_index(drop=True)

,MLO,entry,size
0,Nucleolus,Q0WX57 P40938 Q32P41 Q96TA6 Q8IV42 Q9C0K0 Q9NV...,2015
1,Stress granule,Q9UBW8 P40938 Q9Y678 Q5VYK3 Q00325 Q9BRA2 Q8IY...,1536
2,Nuclear stress body,F5GY65 P26373 Q9NZL9 P35237 B4DEM2 P67809 F8VT...,459
3,P-body,O15063 P67809 Q9Y2H9 Q9BZI7 Q9BZL4 Q8TCB0 P130...,397
4,Nuclear speckle,Q9NRR3 Q9GZU7 Q8TC56 Q86X55 Q16630 Q3ZCQ8 Q9NR...,299
5,Paraspeckle,P62888 Q16630 Q5TGY3 P15924 Q9H0B3 Q96PK6 P067...,252
6,Splicesome,Q2TAY7 Q96NC0 Q08170 Q8WUQ7 Q9BTD8 Q9UBB9 O436...,246
7,Nuclear body,Q08170 Q09028 Q9UBB9 Q9Y6K9 P12947 Q13573 P025...,174
8,PML nuclear body,P23919 P00749 P27694 P61956 Q76I76 Q9NQM4 Q674...,165
9,Cajal body,P61956 P07332 P29376 P08922 P19338 Q6PHR2 Q152...,120


In [177]:
ps_clusters = pd.read_pickle("./saves/INTEGRATE_ProSE80d_pos1neg1/PS_Clusters_basedata.pkl")

In [143]:
# enrichment function used for MLOs and amin feature
def mlo_enrich(Member, Entry):
        members = Member.split(' ')
        entries = Entry.split(' ')
        a = set(members) & set(entries)
        b = set(members) - set(entries)
        c = set(entries) - set(members)
        d = set(genes) - set(members) | set(entries)
        frac = len(a) / len(members)
        from scipy.stats import fisher_exact
        _, pval = fisher_exact([[len(a), len(b)], [len(c), len(d)]])
        return frac, pval

In [ ]:
# mlo_enrich(cl['Members'][0], mloset['entry'][0])

In [236]:
# from collections import defaultdict
pvals, fracs = dict(), dict()
for _, row in tqdm(mloset.iterrows()):
    temp_ls1, temp_ls2 = list(), list()
    mlo, entry = row['MLO'], row['entry']
    for member in ps_clusters['Members']:
        f, p = mlo_enrich(member, entry)
        temp_ls1.append(p)
        temp_ls2.append(f)
    pvals[mlo] = temp_ls1
    fracs[mlo] = temp_ls2

23it [00:14,  1.56it/s]


In [237]:
fracs_df = pd.DataFrame(fracs)
cols = ['Frac ' + c for c in fracs_df.columns]
fracs_df.columns = cols

pvals_df = pd.DataFrame(pvals)
cols = ['FDR ' + c for c in pvals_df.columns]
pvals_df.columns = cols
# FDR correction
from statsmodels.stats.multitest import fdrcorrection
for c in pvals_df.columns:
    _, fdrs = fdrcorrection(pvals_df[c])
    pvals_df[c] = fdrs

enrich_df = pd.concat([ps_clusters, pvals_df, fracs_df], axis=1)

In [266]:
enrich_df.head(2)

,Cluster,Size,Density,Internal weight,External weight,Quality,P-value,Members,FDR PS proteins,Frac PS proteins,...,"Frac PML nuclear body,Nucleolus",Frac Paraspeckle,Frac Polycomb body,Frac Postsynaptic density,Frac SMN complex,Frac Sam68 nuclear body,Frac Spindle apparatus,Frac Splicesome,Frac Stress granule,Frac Viroplasm
0,1,31,0.3021,140.50,758.1,0.1463,0.999874,Q9Y5B9 Q08945 Q96P70 Q9BTT0 P62805 Q01831 P499...,0.001036,0.580645,...,0.0,0.0,0.000000,0.0,0.032258,0.0,0.0,0.0,0.193548,0.0
1,3,24,0.2211,61.03,257.6,0.1665,0.991380,Q9HAH7 Q8IY57 Q86SE9 P61962 Q9BYE7 Q8N488 P783...,0.001381,0.625000,...,0.0,0.0,0.041667,0.0,0.000000,0.0,0.0,0.0,0.041667,0.0


In [239]:
cluster_genes = list(set(enrich_df['Members'].str.cat(sep=' ').split(' ')))

In [267]:
len(cluster_genes)

3278

## Domains enrichment in Pred Clusters

In [149]:
human_domains = pd.read_table("E:/lyz_PhaGAT/代码整理/Fig6/cluster_enrichs/human_domains_Pfam.tsv",sep='\t',header=None)
human_domains.head(1)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,P47989,ff83e5d05d82c0f5b9ba3a0ffa1708b7,1333,Pfam,PF01799,[2Fe-2S] binding domain,87,160,3.500000e-27,T,02-08-2024,IPR002888,[2Fe-2S]-binding,-,-


In [150]:
human_domains = human_domains.iloc[:,[0,3,4,5,6,7,8]]
human_domains.columns = ['Uniprot','DataBase','DomainID','DomainName','Start','End','pvalue']
human_domains['pvalue'] = human_domains['pvalue'].astype(float)
human_domains = human_domains[human_domains['pvalue']<0.05]
human_domains

,Uniprot,DataBase,DomainID,DomainName,Start,End,pvalue
0,P47989,Pfam,PF01799,[2Fe-2S] binding domain,87,160,3.500000e-27
1,Q06278,Pfam,PF01799,[2Fe-2S] binding domain,88,162,2.400000e-26
2,Q96PB1,Pfam,PF07779,10 TM Acyl Transferase domain found in Cas1p,296,775,2.600000e-223
3,Q15029,Pfam,PF16004,116 kDa U5 small nuclear ribonucleoprotein com...,4,110,6.600000e-35
4,P31947,Pfam,PF00244,14-3-3 protein,9,231,1.300000e-98
...,...,...,...,...,...,...,...
47501,P16157,Pfam,PF00791,ZU5 domain,915,1012,5.500000e-25
47502,Q5F1R6,Pfam,PF21884,"Zuotin-like, zuotin homology domain",107,218,1.800000e-12
47503,Q99543,Pfam,PF21884,"Zuotin-like, zuotin homology domain",178,281,9.100000e-42
47504,O43264,Pfam,PF22766,ZW10 C-terminal helical domain,624,778,1.500000e-62


In [163]:
domain_types = np.unique(human_domains['DomainID'].tolist()).tolist()
len(domain_types)

7244

In [164]:
human_domains['Uniprot_s'] = human_domains['Uniprot']+'_'+human_domains.index.astype(str)

In [165]:
domain_genes = human_domains['Uniprot_s'].tolist()
print(len(domain_genes))
add_genes =set(genes) - set(human_domains['Uniprot'].tolist())
print(len(add_genes))
domain_genes = domain_genes+list(add_genes)
len(domain_genes)

41199
987


42186

In [269]:
# enrichment function used for domains and motifs
def feature_enrich_dm(Member, Entry, df,genes1):
        members0 = set(Member.split(' '))
        members1 = members0 & set(df.index.tolist())
        members2 = members0 - set(df.index.tolist())
        
        member1_uds = df.loc[list(members1),:]['Uniprot_s'].tolist()
        members = list(members2)+member1_uds
        
        entries = Entry
        
        a = set(members) & set(entries)
        b = set(members) - set(entries)
        c = set(entries) - set(members)
        d = set(genes1) - set(members) | set(entries)
        frac = len(a) / len(members)
        from scipy.stats import fisher_exact
        _, pval = fisher_exact([[len(a), len(b)], [len(c), len(d)]])
        return frac, pval

In [270]:
human_domains = human_domains.set_index(['Uniprot'])

In [272]:
pvals, fracs = dict(), dict()
for f in tqdm(domain_types):
    entrys = human_domains[human_domains['DomainID']==f]['Uniprot_s'].tolist()
    uniprots = list(human_domains[human_domains['DomainID']==f].index)

    if len(set(cluster_genes) & set(uniprots))>0:
        temp_ls1, temp_ls2 = list(), list()
        for i in range(len(enrich_df)):
            members = enrich_df.loc[i,'Members']
            fra, p = feature_enrich_dm(members, entrys,human_domains,domain_genes)
            temp_ls1.append(p)
            temp_ls2.append(fra)
        pvals[f] = temp_ls1
        fracs[f] = temp_ls2

100%|██████████| 7244/7244 [2:43:36<00:00,  1.36s/it]  


In [273]:
fracs_df = pd.DataFrame(fracs)
cols = ['Frac ' + c for c in fracs_df.columns]
fracs_df.columns = cols

pvals_df = pd.DataFrame(pvals)
cols = ['FDR ' + c for c in pvals_df.columns]
pvals_df.columns = cols
# FDR correction
from statsmodels.stats.multitest import fdrcorrection
for c in pvals_df.columns:
    _, fdrs = fdrcorrection(pvals_df[c])
    pvals_df[c] = fdrs

enrich_df = pd.concat([enrich_df, pvals_df, fracs_df], axis=1)

In [274]:
enrich_df

,Cluster,Size,Density,Internal weight,External weight,Quality,P-value,Members,FDR PS proteins,Frac PS proteins,...,Frac PF22983,Frac PF22984,Frac PF22985,Frac PF22987,Frac PF22988,Frac PF22989,Frac PF23004,Frac PF23011,Frac PF23014,Frac PF23015
0,1,31,0.3021,140.500,758.10,0.14630,0.999874,Q9Y5B9 Q08945 Q96P70 Q9BTT0 P62805 Q01831 P499...,1.036222e-03,0.580645,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1,3,24,0.2211,61.030,257.60,0.16650,0.991380,Q9HAH7 Q8IY57 Q86SE9 P61962 Q9BYE7 Q8N488 P783...,1.380849e-03,0.625000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2,5,82,0.3011,999.900,5236.00,0.15620,0.958322,Q13586 P50402 P78310 Q8N144 Q96RD7 P51151 Q8WX...,7.453350e-05,0.475610,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
3,8,4,0.3167,1.900,22.65,0.05839,0.903035,Q9ULL5 Q8IWW6 Q9UPX8 Q9BYB0,3.507421e-02,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4,9,4,0.3703,2.222,30.80,0.05416,0.992931,P45973 P23497 Q6N043 Q9H5I1,2.109341e-01,0.750000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488,2069,4,0.3148,1.889,55.97,0.02868,0.903035,Q9BW19 Q86XP3 Q9NPA8 P28324,2.109341e-01,0.750000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
489,2078,6,0.3188,4.783,86.66,0.04624,0.967224,P35611 Q8N6D5 Q9UEY8 P35612 P32456 Q5JUQ0,1.952822e-01,0.666667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
490,2079,12,0.3006,19.840,274.80,0.06225,0.999761,Q86V42 Q9HAP6 Q8N3R9 Q9Y2J4 P35580 Q8IY63 P469...,8.810641e-02,0.583333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
491,2082,32,0.3011,149.300,2772.00,0.05002,1.000000,Q96A22 Q03135 P78310 O75955 P06239 P07948 P299...,1.233763e-09,0.812500,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.010309,0.0


In [275]:
enrich_df.to_pickle("./saves/INTEGRATE_ProSE80d_pos1neg1/PS_clusters_with_feature_enrichment.pkl")

## Motifs enrichment in Pred Clusters

In [258]:
human_motifs = pd.read_table("E:/lyz_PhaGAT/代码整理/Fig6/cluster_enrichs/human_motif_FIMO5.5.5.tsv",sep='\t')
human_motifs = human_motifs[human_motifs['p-value']<0.0001]
human_motifs = human_motifs.set_index(['sequence_name']).loc[list(set(human_motifs['sequence_name'].tolist())&set(genes)),:].reset_index().sort_values(by=['sequence_name'])
human_motifs.index=range(len(human_motifs))
human_motifs.head(2)

,sequence_name,motif_id,motif_alt_id,start,stop,strand,score,p-value,q-value,matched_sequence
0,A0A024RBG1,ELME000330,CLV_Separin_Fungi,111.0,117.0,+,8.62887,0.000082,0.906,SVNIGRK
1,A0A087WUL8,ELME000501,LIG_SH3_CIN85_PxpxPR_1,830.0,835.0,+,4.08696,0.000057,0.268,NPPCPR


In [260]:
human_motifs['Uniprot_s'] = human_motifs['sequence_name']+'_'+human_motifs.index.astype(str)

In [261]:
motif_genes = human_motifs['Uniprot_s'].tolist()
print(len(motif_genes))
add_genes =set(genes) - set(human_motifs['sequence_name'].tolist())
print(len(add_genes))
motif_genes = motif_genes+list(add_genes)
len(motif_genes)

216109
262


216371

In [262]:
motif_types = np.unique(human_motifs['motif_id'].tolist()).tolist()
len(motif_types)

192

In [276]:
human_motifs = human_motifs.set_index(['sequence_name'])

In [277]:
pvals, fracs = dict(), dict()
for f in tqdm(motif_types):
    entrys = human_motifs[human_motifs['motif_id']==f]['Uniprot_s'].tolist()
    uniprots = list(human_motifs[human_motifs['motif_id']==f].index)

    if len(set(cluster_genes) & set(uniprots))>0:
        temp_ls1, temp_ls2 = list(), list()
        for i in range(len(enrich_df)):
            members = enrich_df.loc[i,'Members']
            fra, p = feature_enrich_dm(members, entrys,human_motifs,motif_genes)
            temp_ls1.append(p)
            temp_ls2.append(fra)
        pvals[f] = temp_ls1
        fracs[f] = temp_ls2

100%|██████████| 192/192 [1:24:34<00:00, 26.43s/it]


In [278]:
fracs_df = pd.DataFrame(fracs)
cols = ['Frac ' + c for c in fracs_df.columns]
fracs_df.columns = cols

pvals_df = pd.DataFrame(pvals)
cols = ['FDR ' + c for c in pvals_df.columns]
pvals_df.columns = cols
# FDR correction
from statsmodels.stats.multitest import fdrcorrection
for c in pvals_df.columns:
    _, fdrs = fdrcorrection(pvals_df[c])
    pvals_df[c] = fdrs

enrich_df = pd.concat([enrich_df, pvals_df, fracs_df], axis=1)

In [ ]:
#enrich_df.to_pickle("./saves/INTEGRATE_ProSE80d_pos1neg1/PS_clusters_with_feature_enrichment.pkl")

In [280]:
enrich_df

,Cluster,Size,Density,Internal weight,External weight,Quality,P-value,Members,FDR PS proteins,Frac PS proteins,...,Frac ELME000547,Frac ELME000548,Frac ELME000555,Frac ELME000556,Frac ELME000559,Frac ELME000562,Frac ELME000564,Frac ELME000565,Frac ELME000567,Frac ELME000569
0,1,31,0.3021,140.500,758.10,0.14630,0.999874,Q9Y5B9 Q08945 Q96P70 Q9BTT0 P62805 Q01831 P499...,1.036222e-03,0.580645,...,0.005780,0.002890,0.005780,0.005780,0.002890,0.005780,0.005780,0.017341,0.014451,0.000000
1,3,24,0.2211,61.030,257.60,0.16650,0.991380,Q9HAH7 Q8IY57 Q86SE9 P61962 Q9BYE7 Q8N488 P783...,1.380849e-03,0.625000,...,0.005714,0.000000,0.000000,0.000000,0.002857,0.000000,0.008571,0.051429,0.022857,0.020000
2,5,82,0.3011,999.900,5236.00,0.15620,0.958322,Q13586 P50402 P78310 Q8N144 Q96RD7 P51151 Q8WX...,7.453350e-05,0.475610,...,0.005362,0.004468,0.005362,0.006256,0.004468,0.002681,0.002681,0.018767,0.009830,0.003575
3,8,4,0.3167,1.900,22.65,0.05839,0.903035,Q9ULL5 Q8IWW6 Q9UPX8 Q9BYB0,3.507421e-02,1.000000,...,0.004237,0.004237,0.000000,0.000000,0.000000,0.000000,0.008475,0.190678,0.008475,0.033898
4,9,4,0.3703,2.222,30.80,0.05416,0.992931,P45973 P23497 Q6N043 Q9H5I1,2.109341e-01,0.750000,...,0.000000,0.000000,0.000000,0.000000,0.016949,0.000000,0.016949,0.016949,0.016949,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488,2069,4,0.3148,1.889,55.97,0.02868,0.903035,Q9BW19 Q86XP3 Q9NPA8 P28324,2.109341e-01,0.750000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.075000,0.025000,0.000000
489,2078,6,0.3188,4.783,86.66,0.04624,0.967224,P35611 Q8N6D5 Q9UEY8 P35612 P32456 Q5JUQ0,1.952822e-01,0.666667,...,0.013889,0.013889,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.027778
490,2079,12,0.3006,19.840,274.80,0.06225,0.999761,Q86V42 Q9HAP6 Q8N3R9 Q9Y2J4 P35580 Q8IY63 P469...,8.810641e-02,0.583333,...,0.000000,0.000000,0.004367,0.000000,0.013100,0.004367,0.008734,0.043668,0.008734,0.021834
491,2082,32,0.3011,149.300,2772.00,0.05002,1.000000,Q96A22 Q03135 P78310 O75955 P06239 P07948 P299...,1.233763e-09,0.812500,...,0.007194,0.001799,0.008993,0.008993,0.000000,0.003597,0.003597,0.043165,0.012590,0.007194


## Amino features of disordered region enrichment in prey clusters

In [292]:
idp_features = pd.read_pickle("./OtherVisualization/data/human_IDR_sequence_features_202405.pkl").set_index(['uniprot'])
idp_features.head(5)

,range,Sequence,length,A,C,D,E,F,G,H,...,RHK_IVMALP_deltazscores,RHK_FWY_deltazscores,STNQCG_IVMALP_deltazscores,STNQCG_FWY_deltazscores,IVMALP_FWY_deltazscores,WT_pos_block,WT_neg_block,WT_all_block,spacer_block_length,Hydropathy
uniprot,,,,,,,,,,,,,,,,,,,,,
Q96QF7,"(0, 478)",MDGCKKELPRLQEPEEDEDCYILNVQSSSDDTSGSSVARRAPKRQA...,479,0.060543,0.014614,0.229645,0.093946,0.000000,0.025052,0.010438,...,3.972107,0.000000,4.152191,0.000000,0.000000,5.0,8.0,13.0,445.0,2.979541
Q13023,"(178, 753)",QTDILQAFSEETKEGRLDSLTEVDDSGQLTIKCSQNYLSLDCGITA...,576,0.048611,0.017361,0.052083,0.092014,0.015625,0.057292,0.019097,...,1.197245,0.000000,-0.877788,0.000000,0.000000,3.0,1.0,4.0,189.0,3.711632
Q13023,"(1211, 1627)",MSEDALEWDEMDISNKLISLNEESNDLDQELQPVIPSLKLGETSNE...,417,0.026379,0.016787,0.069544,0.076739,0.026379,0.076739,0.033573,...,-0.899457,0.487486,0.659230,0.497874,1.698903,1.0,3.0,4.0,291.0,3.881775
Q13023,"(1641, 2053)",IQSPSEQKIKRSVSDITLQSSSQKMSFTGQMSLDIASSINEDSAAS...,413,0.046005,0.036320,0.089588,0.092010,0.019370,0.038741,0.024213,...,0.849160,0.000000,-1.867838,0.000000,0.000000,1.0,2.0,3.0,213.0,3.702663
Q13023,"(2100, 2318)",PIQLRKGDFYSYLSLSSHDSDCGEVTNYIEEKSSTPLPLDTTDSGL...,219,0.082192,0.031963,0.077626,0.114155,0.027397,0.050228,0.027397,...,0.621740,0.000000,-0.841309,0.000000,0.000000,1.0,1.0,2.0,149.0,3.709132


In [293]:
mlo_features = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'acid', 'basic', 'nonpolar', 'hydrophilic', 'aromatic', 'charged', 'nu', 'betaTurn', 'DE_omegazscores', 'RHK_omegazscores', 'DERHK_omegazscores', 'STNQCG_omegazscores', 'IVMALP_omegazscores', 'FWY_omegazscores', 'NQ_omegazscores', 'SG_omegazscores', 'SR_omegazscores', 'A_omegazscores', 'C_omegazscores', 'D_omegazscores', 'E_omegazscores', 'F_omegazscores', 'G_omegazscores', 'H_omegazscores', 'I_omegazscores', 'K_omegazscores', 'L_omegazscores', 'M_omegazscores', 'N_omegazscores', 'P_omegazscores', 'Q_omegazscores', 'R_omegazscores', 'S_omegazscores', 'T_omegazscores', 'V_omegazscores', 'W_omegazscores', 'Y_omegazscores', 'DE_RHK_deltazscores', 'DE_STNQCG_deltazscores', 'DE_IVMALP_deltazscores', 'DE_FWY_deltazscores', 'RHK_STNQCG_deltazscores', 'RHK_IVMALP_deltazscores', 'RHK_FWY_deltazscores', 'STNQCG_IVMALP_deltazscores', 'STNQCG_FWY_deltazscores', 'IVMALP_FWY_deltazscores', 'WT_pos_block', 'WT_neg_block', 'WT_all_block', 'Hydropathy','NCPR']

In [294]:
idp_features = idp_features.loc[[i for i in genes if i in idp_features.index],mlo_features]
idp_features

,A,C,D,E,F,G,H,I,K,L,...,RHK_IVMALP_deltazscores,RHK_FWY_deltazscores,STNQCG_IVMALP_deltazscores,STNQCG_FWY_deltazscores,IVMALP_FWY_deltazscores,WT_pos_block,WT_neg_block,WT_all_block,Hydropathy,NCPR
uniprot,,,,,,,,,,,,,,,,,,,,,
A0A087WUL8,0.022289,0.042403,0.086165,0.140799,0.014406,0.057353,0.006252,0.019571,0.055993,0.113618,...,14.006156,19.175716,-2.581196,-5.189864,0.214555,14.0,32.0,46.0,3.531068,-0.096222
A0A087X1G2,0.098446,0.036269,0.046632,0.041451,0.041451,0.088083,0.015544,0.010363,0.015544,0.051813,...,-0.972567,-1.603406,0.362147,1.467279,0.403002,0.0,0.0,0.0,3.752850,0.015544
A0A096LP49,0.104317,0.032374,0.053957,0.046763,0.007194,0.075540,0.028777,0.007194,0.075540,0.064748,...,0.266269,0.000000,1.640087,0.000000,0.000000,1.0,0.0,1.0,3.605036,0.061151
A0A096LP49,0.088608,0.029536,0.021097,0.050633,0.021097,0.092827,0.021097,0.004219,0.033755,0.050633,...,-1.353523,-0.129248,-1.876398,-1.603597,-1.337016,0.0,0.0,0.0,3.337131,0.084388
A0A096LP49,0.095238,0.023810,0.033333,0.047619,0.019048,0.114286,0.028571,0.009524,0.023810,0.080952,...,0.426318,1.068689,-0.946103,-1.320900,0.874632,0.0,0.0,0.0,3.977619,0.009524
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Q9Y6Y1,0.121212,0.010101,0.040404,0.090909,0.040404,0.020202,0.000000,0.030303,0.020202,0.090909,...,0.000000,0.000000,-1.180884,1.027701,-1.000106,0.0,0.0,0.0,4.275758,-0.090909
Q9Y6Y1,0.014706,0.058824,0.029412,0.044118,0.029412,0.044118,0.029412,0.029412,0.102941,0.102941,...,-0.158706,-0.184224,2.795428,1.645924,-0.169248,0.0,0.0,0.0,4.161765,0.058824
Q9Y6Y8,0.072519,0.003817,0.019084,0.019084,0.049618,0.087786,0.011450,0.026718,0.011450,0.061069,...,0.000000,0.000000,0.333603,1.240379,0.298752,0.0,1.0,1.0,4.107634,-0.011450


In [286]:
15225/10

1522.5

In [295]:
all_features_most = {}
for f in mlo_features:
    all_features_most[f] = list(set(list(idp_features[idp_features[f]!='_'].sort_values(by=[f],ascending=False).iloc[:1523,:].index)))

In [299]:
def feature_enrich(Member, Entry):
        members = Member.split(' ')
        entries = Entry
        a = set(members) & set(entries)
        b = set(members) - set(entries)
        c = set(entries) - set(members)
        d = set(genes) - set(members) | set(entries)
        frac = len(a) / len(members)
        from scipy.stats import fisher_exact
        _, pval = fisher_exact([[len(a), len(b)], [len(c), len(d)]])
        return frac, pval

In [300]:
pvals, fracs = dict(), dict()
for f in tqdm(mlo_features):
    entrys = all_features_most[f]
    temp_ls1, temp_ls2 = list(), list()
    for i in range(len(enrich_df)):
        members = enrich_df.loc[i,'Members']
        fra, p = feature_enrich(members, entrys)
        temp_ls1.append(p)
        temp_ls2.append(fra)
    pvals[f] = temp_ls1
    fracs[f] = temp_ls2

100%|██████████| 72/72 [00:59<00:00,  1.22it/s]


In [301]:
fracs_df = pd.DataFrame(fracs)
cols = ['Frac ' + c for c in fracs_df.columns]
fracs_df.columns = cols

pvals_df = pd.DataFrame(pvals)
cols = ['FDR ' + c for c in pvals_df.columns]
pvals_df.columns = cols
# FDR correction
from statsmodels.stats.multitest import fdrcorrection
for c in pvals_df.columns:
    _, fdrs = fdrcorrection(pvals_df[c])
    pvals_df[c] = fdrs

enrich_df = pd.concat([enrich_df, pvals_df, fracs_df], axis=1)

In [ ]:
#enrich_df.to_pickle("./saves/INTEGRATE_ProSE80d_pos1neg1/PS_clusters_with_feature_enrichment.pkl")

In [303]:
enrich_df

,Cluster,Size,Density,Internal weight,External weight,Quality,P-value,Members,FDR PS proteins,Frac PS proteins,...,Frac RHK_IVMALP_deltazscores,Frac RHK_FWY_deltazscores,Frac STNQCG_IVMALP_deltazscores,Frac STNQCG_FWY_deltazscores,Frac IVMALP_FWY_deltazscores,Frac WT_pos_block,Frac WT_neg_block,Frac WT_all_block,Frac Hydropathy,Frac NCPR
0,1,31,0.3021,140.500,758.10,0.14630,0.999874,Q9Y5B9 Q08945 Q96P70 Q9BTT0 P62805 Q01831 P499...,1.036222e-03,0.580645,...,0.129032,0.193548,0.096774,0.064516,0.032258,0.516129,0.387097,0.548387,0.032258,0.516129
1,3,24,0.2211,61.030,257.60,0.16650,0.991380,Q9HAH7 Q8IY57 Q86SE9 P61962 Q9BYE7 Q8N488 P783...,1.380849e-03,0.625000,...,0.250000,0.083333,0.041667,0.000000,0.166667,0.291667,0.041667,0.166667,0.000000,0.166667
2,5,82,0.3011,999.900,5236.00,0.15620,0.958322,Q13586 P50402 P78310 Q8N144 Q96RD7 P51151 Q8WX...,7.453350e-05,0.475610,...,0.109756,0.085366,0.121951,0.097561,0.060976,0.060976,0.195122,0.097561,0.073171,0.097561
3,8,4,0.3167,1.900,22.65,0.05839,0.903035,Q9ULL5 Q8IWW6 Q9UPX8 Q9BYB0,3.507421e-02,1.000000,...,0.250000,0.000000,0.250000,0.000000,0.000000,0.250000,0.500000,0.500000,0.000000,0.250000
4,9,4,0.3703,2.222,30.80,0.05416,0.992931,P45973 P23497 Q6N043 Q9H5I1,2.109341e-01,0.750000,...,0.250000,0.000000,0.000000,0.250000,0.000000,0.250000,0.500000,0.250000,0.000000,0.250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488,2069,4,0.3148,1.889,55.97,0.02868,0.903035,Q9BW19 Q86XP3 Q9NPA8 P28324,2.109341e-01,0.750000,...,0.250000,0.250000,0.000000,0.000000,0.000000,0.000000,0.250000,0.250000,0.000000,0.250000
489,2078,6,0.3188,4.783,86.66,0.04624,0.967224,P35611 Q8N6D5 Q9UEY8 P35612 P32456 Q5JUQ0,1.952822e-01,0.666667,...,0.500000,0.000000,0.000000,0.000000,0.000000,0.500000,0.333333,0.333333,0.000000,0.000000
490,2079,12,0.3006,19.840,274.80,0.06225,0.999761,Q86V42 Q9HAP6 Q8N3R9 Q9Y2J4 P35580 Q8IY63 P469...,8.810641e-02,0.583333,...,0.000000,0.250000,0.166667,0.250000,0.083333,0.083333,0.166667,0.166667,0.083333,0.083333
491,2082,32,0.3011,149.300,2772.00,0.05002,1.000000,Q96A22 Q03135 P78310 O75955 P06239 P07948 P299...,1.233763e-09,0.812500,...,0.312500,0.281250,0.281250,0.218750,0.312500,0.187500,0.156250,0.156250,0.125000,0.062500
